<a href="https://colab.research.google.com/github/sanal-m-s/gesture-control-using-ml/blob/main/code6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from function import *
from keras.models import model_from_json
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard
import os
import time
import cv2
import numpy as np

buttonPressed = False
buttonCounter = 0
buttonDelay = 30

json_file = open("model.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("model.h5")

colors = []
for i in range(0, 20):
    colors.append((245, 117, 16))
print(len(colors))

# 1. New detection variables
sequence = []
sentence = []
accuracy = []
predictions = []
threshold = 0.8

# Add a variable to keep track of the current index of the displayed image
current_image_index = 0

# Add variables to track the position of the red circle marker
red_circle_position = (0, 0)
red_circle_radius = 20

# Add variables to track drawing
drawing = False
prev_point = None

cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture("https://192.168.43.41:8080/video")
# Set mediapipe model
with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
    # Set the path to the Presentation folder
    presentation_folder = "Presentation"

    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        cropframe = frame[40:400, 0:300]
        frame = cv2.rectangle(frame, (0, 40), (300, 400), 255, 2)
        image, results = mediapipe_detection(cropframe, hands)

        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        try:
            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                predicted_action = actions[np.argmax(res)]
                print(predicted_action)
                predictions.append(np.argmax(res))

                # 3. Viz logic
                if np.unique(predictions[-10:])[0] == np.argmax(res):
                    if res[np.argmax(res)] > threshold:
                        if len(sentence) > 0:
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                                accuracy.append(str(res[np.argmax(res)] * 100))
                        else:
                            sentence.append(actions[np.argmax(res)])
                            accuracy.append(str(res[np.argmax(res)] * 100))

                if len(sentence) > 1:
                    sentence = sentence[-1:]
                    accuracy = accuracy[-1:]

                # Viz probabilities
                # frame = prob_viz(res, actions, frame, colors, threshold)
                # Control images in the Presentation folder based on the recognized gesture
                presentation_images = os.listdir(presentation_folder)

                if predicted_action == "A":
                    # Code to navigate to the next image
                    current_image_index += 1
                    buttonPressed = True
                    if current_image_index >= len(presentation_images):
                        current_image_index = 0

                    if len(presentation_images) > current_image_index:
                        image_path = os.path.join(presentation_folder, presentation_images[current_image_index])
                        presentation_image = cv2.imread(image_path)
                        cv2.imshow('Presentation Image', presentation_image)
                        time.sleep(1)

                elif predicted_action == "B":
                    # Code to navigate to the previous image
                    current_image_index -= 1
                    buttonPressed = True
                    if current_image_index < 0:
                        current_image_index = len(presentation_images) - 1

                    if len(presentation_images) > current_image_index:
                        image_path = os.path.join(presentation_folder, presentation_images[current_image_index])
                        presentation_image = cv2.imread(image_path)
                        cv2.imshow('Presentation Image', presentation_image)
                        time.sleep(1)

                elif predicted_action == "C":
                    # Code to track hand landmarks and update the position of the red circle marker
                    hand_landmarks = results.multi_hand_landmarks[0] if results.multi_hand_landmarks else None

                    if hand_landmarks:
                        # Get the coordinates of the tip of the index finger
                        index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                        index_finger_x, index_finger_y = int(index_finger_tip.x * frame.shape[1]), int(
                            index_finger_tip.y * frame.shape[0])

                        # Update the position of the red circle marker
                        red_circle_position = (index_finger_x, index_finger_y)

                    # Draw the red circle marker on the current slide
                    if len(presentation_images) > current_image_index:
                        image_path = os.path.join(presentation_folder, presentation_images[current_image_index])
                        presentation_image = cv2.imread(image_path)
                        cv2.circle(presentation_image, red_circle_position, red_circle_radius, (0, 0, 255), -1)
                        cv2.imshow('Presentation Image', presentation_image)

                elif predicted_action == "D":
                    # Code to draw on the current presentation slide
                    hand_landmarks = results.multi_hand_landmarks[0] if results.multi_hand_landmarks else None

                    if hand_landmarks:
                        index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                        x, y = int(index_finger_tip.x * frame.shape[1]), int(index_finger_tip.y * frame.shape[0])

                        if prev_point is not None:
                            cv2.line(presentation_image, prev_point, (x, y), (0, 255, 0), 5)

                        prev_point = (x, y)

                    # Show the updated presentation image
                    cv2.imshow('Presentation Image', presentation_image)

                elif predicted_action == "E":
                    # Code to remove the drawn content on the slide
                    presentation_image = cv2.imread(image_path)
                    cv2.imshow('Presentation Image', presentation_image)
                    drawing_canvas = np.zeros_like(presentation_image)

        except Exception as e:
            # Handle exceptions
            pass

            cv2.rectangle(frame, (0, 0), (300, 40), (245, 117, 16), -1)
            cv2.putText(frame, "Output: -" + ' '.join(sentence) + ''.join(accuracy), (3, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Show to screen
            cv2.imshow('OpenCV Feed', frame)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()